In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Problem statement

For many ML-models, but not all, it is required that all the input features are numerical - since the values they have are being used in mathematical operations. 
However, in the previous lab with the used cars dataset, we had to omit all features that were not already numerical since we hadn't yet learned how to deal with them.

In this lab, we'll learn of a technique called One Hot Encoding, which is a method of transforming what we call *categorical* features into numerical ones.

**Let's randomly generate a dataset to work with here**

In [ ]:
ages = np.random.randint(20, 60, size=10)
genders = np.random.choice(['male', 'female', 'other'], size=10)
salaries = np.random.randint(30000, 80000, size=10)

data = {'Age': ages, 'Gender': genders, 'Salary': salaries}
df = pd.DataFrame(data)

df

In the dataset above, the column Gender is called *categorical*.

A categorical feature is one in which we can't say that one value is more or less than another one. 

In this case, male is neither more or less than female, i.e., we can't say that male < female or vice-versa.

Some other examples of categorical data are hair color (red, blonde, or black), political affiliation (republican, democrat, or other), and favourite food (sushi, kebab, or vegetables). Again, these are considered categorical because one is not more than the other.

In contrast, the columns Age and Salary are definitely not categorical since you can directly compare them. A salary of 60000 > 30000, and conversely 39 is objectively older than 26.

**One Hot Encoding**

A very simple way of dealing with categorical features is simply to create boolean columns, one for each of the values of our categorical column. This is called one hot encoding. We can do this in several ways, and Pandas has a pretty neat function for it aswell.

In [ ]:
df

In [ ]:
# 1. we call on the get_dummies function
# 2. we give it the dataset as the first argument
# 3. We tell it which columns we want to one hot encode

pd.get_dummies(df, columns = ['Gender']) 

In [ ]:
# 1. we call on the get_dummies function
# 2. we give it the dataset as the first argument
# 3. We tell it which columns we want to one hot encode
# 4. In addition, we also let the function know that we want 1/0 and not True/False

pd.get_dummies(df, columns = ['Gender'], dtype=int)

The result is a new dataframe, with categorical features transformed into numerical ones that we can use for the models that require it! 

If now use these features to try to predict the salary for a given employee, we might have something like this (for a linear model)

$$ salary = w_3 \cdot (Age) + w_2 \cdot (Gender\ female) + w_1 \cdot (Gender\ male) + w_0 $$

**A word of caution**

However, you must be careful to not fall into the trap of believing that numerical columns can't be categorical! 

Take this for example:

In [ ]:
ages = np.random.randint(20, 60, size=10)
genders = np.random.choice(['male', 'female'], size=10)
employee_ids = np.random.randint(200, 600, size=10)
salaries = np.random.randint(30000, 80000, size=10)

data = {'Employee ID': employee_ids, 'Age': ages, 'Gender': genders, 'Salary': salaries}
df = pd.DataFrame(data)

df

In [ ]:
df.info()

Obviously, Employee ID is a numerical column and we can technically do mathematical comparisons between each Employee ID.

532 > 220. Obviously.

However, imagine using column, as is, as a feature for, say, a linear model that predicts salary. Then it might look like this:

$$ salary = w_2 \cdot (Employee\ ID) + w_1 \cdot (Age) + w_0 $$

According to this, a higher Employee ID contributes more to the predicted salary than a lower Employee ID. That doesn't seem to make any sense at all.

In fact, the column Employee ID is also a categorical feature, a *numerical categorical* feature. We could technically do comparisons between its values, but we really shouldn't - since it doesnt make sense. 

We can treat numerical categorical columns just like we did with ordinary categorical features - we'll one hot encode them.



In [ ]:
df.info()

In [ ]:
pd.get_dummies(df, columns = ['Gender', 'Employee ID'], dtype=int)  # note that we can one hot encode several columns at the same time

För mer information, läs [dokumentationen!](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

In [ ]:
ages = np.random.randint(20, 60, size=10)
genders = np.random.choice(['male', 'female', 'other'], size=10)
education = np.random.choice(['high school', 'bachelor', 'phd'], size=10)
salaries = np.random.randint(30000, 80000, size=10)

data = {'Age': ages, 'Gender': genders, 'Education': education, 'Salary': salaries}
df = pd.DataFrame(data)

df

In [ ]:
def edu_mapper(value):

    if value=='phd':
        return 3
    elif value=='bachelor': 
        return 2
    else:
        return 1

In [ ]:
numerical_education = [edu_mapper(level) for level in df['Education']]

In [ ]:
df['New Education'] = numerical_education

In [ ]:
df

$$ salary = w_2 \cdot (Age) + w_1 \cdot (New\ Education) + w_0$$

Ni kan göra som ovan, men ni kan också givetvis köra dummy variables ändå. Trots att Education inte är kategorisk

In [ ]:
ages = np.random.randint(20, 60, size=10)
genders = np.random.choice(['male', 'female', 'other'], size=10)
education = np.random.choice(['high school', 'bachelor', 'phd'], size=10)
salaries = np.random.randint(30000, 80000, size=10)

data = {'Age': ages, 'Gender': genders, 'Education': education, 'Salary': salaries}
df = pd.DataFrame(data)


pd.get_dummies(df, columns = ['Education'], dtype=int)  # note that we can one hot encode several columns at the same time

$$ salary = w_4 \cdot (Age) + w_3 \cdot (Education\ bachelor) + w_2 \cdot (Education\ high\ school) + w_1 \cdot (Education\ phd) + w_0$$

**The curse of dimensionality**

One Hot Encoding is great, but you should be careful using it. If the categorical column you transform has too many values, it will translate to equally many new columns. This means that the dimensionality of your features increases (each feature is one dimension). As you increase the number of features, it also turns out that it gets more and more difficult to fit a model to it - unless you have a big enough data set!

Think about it. If you have a very small data set (10 samples), and 1000 features - you're not going to be able to make any sense of the underlying process you're trying to model.

However, if you have a relatively big data set (1M samples), and 1000 features - you might be able to make sense of the underlying process and produce a good model.

**conclusion**

One Hot Encoding is great, use it. But be careful. If the column your transforming has *alot* of possible values (say, perhaps over 20 or 30), it might case issues if you don't also have a correspondingly good amount of data.

Clear warning sign: if the number of features exceed the number of training samples, you're most likely in trouble.

What you want is for your training samples >> number of features